In [1]:
import tensorflow as tf
import numpy as np
import gym
from tensorflow import keras
import math

In [2]:
# env = gym.make('BipedalWalker-v3')
# #reset inizializza environment
# #observation contiene le informazioni relative allo stato
# observation = env.reset()
# print("Observation: ", observation)
# #numero di azioni che è possibile eseguire
# print("Action space: ",env.action_space)
# #numero di osservazioni che otteniamo dallo stato,
# #ad esempio nel primo stato otteniamo quelle stampate precedentemente
# print("Observation: ", env.observation_space)

Volgio utilizzare una DQN per addestrare il mio agente in grado di imparare a camminare. Per questo metodo avrò bisogno:
1. Epsilon greedy selection rule: per selezionare l'azione, con probabilità epsilon andrò a selezionare un'azione random, nel resto dei casi l'azione sarà calcolata da una Neural Network.
2. Neural Network: la rete avrà un input layer con 160 nodi (il numero di observation fornite dalo state) e nell'output layer vi saranno 4 nodi, pari al numero di azioni da eseguire, per ogni nodo vi sarà un valore che rappresenta il Q-value, e noi saremo interessati al valore massimo.
3. Nel caso della DQN, per eseguire l'update dei parametri della rete, (Weight of the connections), andremo ad utilizzare una experience replay.
4. Experience Replay: non è altro che un batch in cui andramo ad inserire tutti gli stati visitati con le relative informazioni fornite da env.step(action)
5. Per eseguire l'update andremo, dopo un certo numero di timestep ad eseguire l'update tramite il policy gradient method.
6. Devo discretizzare l'uscita, ogni nodo corrisponde a 0.1 (-1, -0.9, .. 1 per la prima azione e così via)

In [3]:
NUM_EPISODE = 200
NUM_TIMESTEP = 200
LEARNING_RATE = 0.001 
DISCOUNT_FACTOR = 0.9

environment = gym.make('BipedalWalker-v3')
#retrieve possible action that I can apply
INPUT = [environment.observation_space.shape[0]]
n_outputs = [environment.action_space.shape[0]]

optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss_function = keras.losses.mean_squared_error

model = keras.Sequential([
    keras.layers.Dense(50,activation="relu", input_shape=INPUT),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(21*n_outputs[0])
])

model.compile(optimizer=optimizer, loss=loss_function)
model.summary()

c:\users\perro\appdata\local\programs\python\python38\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1250      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 84)                4284      
Total params: 8,084
Trainable params: 8,084
Non-trainable params: 0
_________________________________________________________________


In [4]:
dictionarie =  {}
start =  - 1.0
for i in range(84):
    dictionarie[i] = start
    if start == 1:
        start = -1
    else:
        start = round(start + 0.1,1)
print(dictionarie)

{0: -1.0, 1: -0.9, 2: -0.8, 3: -0.7, 4: -0.6, 5: -0.5, 6: -0.4, 7: -0.3, 8: -0.2, 9: -0.1, 10: 0.0, 11: 0.1, 12: 0.2, 13: 0.3, 14: 0.4, 15: 0.5, 16: 0.6, 17: 0.7, 18: 0.8, 19: 0.9, 20: 1.0, 21: -1, 22: -0.9, 23: -0.8, 24: -0.7, 25: -0.6, 26: -0.5, 27: -0.4, 28: -0.3, 29: -0.2, 30: -0.1, 31: 0.0, 32: 0.1, 33: 0.2, 34: 0.3, 35: 0.4, 36: 0.5, 37: 0.6, 38: 0.7, 39: 0.8, 40: 0.9, 41: 1.0, 42: -1, 43: -0.9, 44: -0.8, 45: -0.7, 46: -0.6, 47: -0.5, 48: -0.4, 49: -0.3, 50: -0.2, 51: -0.1, 52: 0.0, 53: 0.1, 54: 0.2, 55: 0.3, 56: 0.4, 57: 0.5, 58: 0.6, 59: 0.7, 60: 0.8, 61: 0.9, 62: 1.0, 63: -1, 64: -0.9, 65: -0.8, 66: -0.7, 67: -0.6, 68: -0.5, 69: -0.4, 70: -0.3, 71: -0.2, 72: -0.1, 73: 0.0, 74: 0.1, 75: 0.2, 76: 0.3, 77: 0.4, 78: 0.5, 79: 0.6, 80: 0.7, 81: 0.8, 82: 0.9, 83: 1.0}


In [5]:
def get_max_Q_values_each_action(Q_values):
    best_index_action = []
    for i in range(4):
        print("Range: from {} to {} ".format(21*i, 21*(i+1)) )
        best_index_action.append(np.argmax(Q_values[0][21*i:21*(i+1)]) + 21*i)
        print(best_index_action)
    return np.array(best_index_action, dtype=np.int16)

In [9]:
#I use DQN so I need Q-learning and Neural Network
#For Q-learning I use epsilon greedy seletion rule, I need a function to do that
def epsilon_greedy(state, epsilon):
    action = [0,0,0,0]
    if np.random.rand() < epsilon:
        #I pick a random action
        return np.random.uniform(low=-1, high=1, size=4)
    else:
        #Starting from the state and using the network I generate the qvalues
        #associated to the action and pick the highest
        Q_values = model.predict(state[np.newaxis])
        print(Q_values)
        index_max_qvalues = get_max_Q_values_each_action(Q_values)
        #varie alternative
        for i in range(4):
            action[i] = dictionarie[index_max_qvalues[i]]
        print("Action ", action)
        return action

In [10]:
def perform_action(observation, epsilon, environment):
    #1. seleziono azione con epsilon greedy policy
    action = epsilon_greedy(observation, epsilon)
    state, reward, done, info = environment.step(action)
    #2. aggiungo nel replpay buffer
    return state, reward, done, info

In [ ]:
for episode in range(NUM_EPISODE):
    print("Current episode: ",episode)
    state = environment.reset()
    for timestep in range(NUM_TIMESTEP):
        epsilon = 0.1
        state, reward, done, info = perform_action(state, epsilon, environment)
        environment.render()
    print("Episode {} completed.".format(episode))

Current episode:  0
[[-0.06284899  0.07775752  0.03449251 -0.08162788  0.00888606 -0.25299066
  -0.03144601 -0.11282938  0.04564818  0.02267771 -0.1650898   0.363233
  -0.01055746  0.3124316   0.03808935  0.2671237  -0.15244645 -0.20751283
  -0.05889032  0.1145708   0.13721772 -0.24386722  0.43016088  0.07697242
  -0.13938236 -0.2715665  -0.09355113  0.14730169  0.2525655  -0.10032339
  -0.26624334 -0.13951962 -0.36706963  0.0878785  -0.03378243 -0.2560172
   0.14212747  0.13995527 -0.17742398  0.1065532  -0.0950695  -0.08650054
   0.0129433  -0.14115658  0.21915159  0.17950942  0.021203   -0.00813634
  -0.09986494 -0.28674534 -0.05993032  0.07828594  0.07130808  0.3089207
   0.04056658 -0.16298842  0.22111173 -0.29539642  0.3267999  -0.29386094
   0.0017515  -0.22451809  0.3853053   0.16107357  0.13234329 -0.18346131
  -0.06585769 -0.2132459   0.17460285 -0.1381607  -0.02932173  0.05143355
   0.3843756  -0.24777447 -0.08798928 -0.05037573  0.07037229  0.14518756
  -0.1210881   0.13564

[[-0.27531326  0.25276664 -0.19491315 -0.12130263  0.05160585 -0.33567002
   0.29816085 -0.27433246  0.02283811  0.18074438 -0.09414422  0.30056372
  -0.22762857  0.30473846  0.16017678  0.28666288  0.06028169 -0.20517343
  -0.02272971  0.04257368  0.19256417 -0.23455423  0.36096957  0.17443159
  -0.34752768 -0.25169733 -0.22102442  0.2051387  -0.09090801 -0.11090624
  -0.17768107 -0.24580486 -0.2798912  -0.06742917 -0.18128935 -0.29356253
   0.21876167 -0.09564752 -0.2389122   0.29650915  0.04360371  0.09367456
   0.07579163 -0.02518334  0.16989952  0.06533518 -0.10232343  0.01865835
  -0.1606083  -0.05103498  0.20419885  0.11276762  0.16541912  0.22605793
  -0.03632459 -0.15940227  0.37837914 -0.20537801  0.44329962 -0.26369268
  -0.09417309 -0.30011225  0.27517474  0.17795394  0.0197816  -0.00862893
  -0.12962256 -0.22597326  0.07879329 -0.20626897 -0.20048858  0.11537377
   0.33687103 -0.3108092  -0.11357329 -0.12404975  0.14821304 -0.04785393
  -0.345504   -0.002362   -0.0206691  

[[-0.0745487   0.17244972  0.02102185 -0.01099275  0.0577055  -0.329134
   0.16085903 -0.17168775 -0.00115916  0.07503235 -0.13207035  0.2709037
  -0.2625515   0.33001313  0.05282202  0.350597   -0.02699954 -0.3373513
  -0.07498084  0.03118636  0.23020825 -0.21689725  0.34381506  0.14844063
  -0.18979606 -0.1461231  -0.08978073  0.02076996 -0.02898508 -0.049787
  -0.13455728 -0.16218726 -0.15593939  0.06005323 -0.05007999 -0.23300877
   0.18969321  0.07312619 -0.13588648  0.21495412 -0.07186601  0.05872083
   0.10299084 -0.00954614  0.14148985  0.18343182 -0.12669124  0.04144252
  -0.03345107 -0.05322709  0.13329875  0.01444092  0.14634147  0.23550972
  -0.04589318 -0.14179963  0.16534328 -0.14676657  0.21884099 -0.2775545
  -0.03352104 -0.36426157  0.22930492  0.0930134   0.08128687 -0.16030237
   0.0007562  -0.22521085  0.01640253 -0.14009194 -0.08852236  0.0753084
   0.29489008 -0.2744429  -0.01046464 -0.05575829  0.12927648 -0.00887443
  -0.21136189  0.11422031  0.11941752  0.21958

  -0.20932989  0.11750127  0.11738627  0.21573383  0.0362491   0.08026309]]
Range: from 0 to 21 
[15]
Range: from 21 to 42 
[15, 22]
Range: from 42 to 63 
[15, 22, 53]
Range: from 63 to 84 
[15, 22, 53, 72]
Action  [0.5, -0.9, 0.1, -0.1]
[[-5.25897257e-02  1.42609328e-01  3.69368345e-02 -2.58263797e-02
   4.16062064e-02 -3.11867893e-01  1.42883152e-01 -1.60945952e-01
  -9.22913849e-03  8.30249116e-02 -1.41535908e-01  2.89847255e-01
  -2.44659871e-01  3.31262290e-01  4.75608706e-02  3.57446551e-01
  -4.22997214e-02 -3.18299979e-01 -6.61413521e-02  4.87594232e-02
   2.06500530e-01 -2.25555107e-01  3.63759041e-01  1.05015144e-01
  -1.94350541e-01 -1.57407999e-01 -7.21473023e-02  1.67738125e-02
   2.29752436e-02 -5.99552616e-02 -1.53365493e-01 -1.61506444e-01
  -1.74703687e-01  8.02294686e-02 -3.54999602e-02 -2.34174415e-01
   2.01358527e-01  1.05755202e-01 -1.40193403e-01  1.88432634e-01
  -7.65638128e-02  3.78507823e-02  9.98018086e-02 -3.89290303e-02
   1.57226041e-01  1.69672310e-01 -1

  -0.19314988  0.08090575  0.07770628  0.16935669 -0.00470448  0.10863849]]
Range: from 0 to 21 
[11]
Range: from 21 to 42 
[11, 22]
Range: from 42 to 63 
[11, 22, 58]
Range: from 63 to 84 
[11, 22, 58, 72]
Action  [0.1, -0.9, 0.6, -0.1]
[[-0.06789564  0.22416067 -0.07231066  0.02349876  0.05901042 -0.32410678
   0.07183377 -0.18926091  0.06560981  0.02234055 -0.10429282  0.29522625
  -0.21791491  0.26330096  0.0885497   0.24970327 -0.03286804 -0.30200922
  -0.12525398  0.03712413  0.14350149 -0.23056573  0.2203621   0.17602164
  -0.2413306  -0.13276723 -0.19615386  0.12258881  0.07251749 -0.08197115
  -0.06106778 -0.0915051  -0.18178663  0.03810306 -0.02317505 -0.22167924
   0.07585382  0.05751894 -0.19962461  0.14589748 -0.01455824  0.07018808
   0.09294828 -0.10459648  0.08166666  0.1322457  -0.05924727 -0.10252622
  -0.04552621 -0.10533817  0.0255533  -0.04467347  0.11219538  0.24380657
  -0.11912085 -0.20019521  0.22321795 -0.23314801  0.29195514 -0.289096
  -0.0345603  -0.2008614

...

[[-0.11825819  0.21775062 -0.04623336  0.01921213 -0.00941507 -0.3156924
   0.05231379 -0.23611157 -0.02186276  0.03033474 -0.06641008  0.24193409
  -0.20895913  0.2499639   0.03111309  0.21608624 -0.01461926 -0.26883432
  -0.10428415  0.02101794  0.18558611 -0.27889615  0.21101262  0.15374246
  -0.27624536 -0.16318676 -0.13123323  0.0819813   0.05687344 -0.18382174
  -0.08384856 -0.08274899 -0.18712324  0.01837075 -0.02350003 -0.2344214
   0.12648684  0.06311397 -0.20113824  0.09172016 -0.05130522  0.0798961
   0.0936141  -0.0780696   0.10619604  0.10426704 -0.06553643 -0.0588425
  -0.08194634 -0.087025   -0.01684543 -0.04159702  0.07158394  0.29778194
  -0.03756866 -0.15193346  0.19686587 -0.25280267  0.2783556  -0.17370799
  -0.02228436 -0.17990007  0.17275283  0.14902851  0.02998754 -0.13974734
  -0.02039735 -0.15317917 -0.01255668 -0.04193457 -0.0886454   0.07966706
   0.27706385 -0.18773569 -0.0907701  -0.15060544  0.13842005 -0.01229396
  -0.17443736  0.05809515  0.06684009  0.1

[[-1.20068453e-01  6.90542087e-02  1.40947551e-02 -9.03972760e-02
  -1.64894655e-01 -2.61684835e-01 -2.52106249e-01 -7.58936480e-02
  -9.82956439e-02  4.42446060e-02 -9.57788825e-02  1.17799945e-01
  -4.87480499e-02  2.61356920e-01  8.35536346e-02  1.23807117e-01
  -2.61218008e-02 -2.46755078e-01  1.47624575e-02  1.09207027e-01
   1.47028536e-01 -3.65489453e-01  1.17860079e-01  7.35859498e-02
  -8.39912444e-02 -1.69534147e-01 -8.51503536e-02 -6.97855875e-02
   3.48323286e-01 -2.73179352e-01  2.52152234e-03 -1.31861001e-01
  -1.71925351e-01  1.23329043e-01  1.23702995e-01 -2.88305581e-01
  -2.54280176e-02  9.90834087e-05 -2.50711024e-01  3.73666137e-02
   2.42849067e-03  8.36412087e-02 -3.10861561e-02 -1.68156400e-01
   8.43665376e-02  1.24486491e-01  5.88478073e-02 -1.82187915e-01
  -1.50827944e-01 -2.32599284e-02 -1.84355408e-01 -1.04875848e-01
   4.09812629e-02  2.58195400e-01 -5.75991645e-02 -2.02085823e-01
   1.67076424e-01 -4.05206233e-01  3.22263122e-01 -8.34300742e-02
   8.61661

[[-0.15488164  0.04287115  0.00198752 -0.12089024 -0.17494021 -0.23024556
  -0.2339478  -0.0766464  -0.16110194  0.08521075 -0.05965108  0.10213666
  -0.03768836  0.27827132  0.07555977  0.1039928  -0.01631515 -0.21376805
  -0.00372602  0.14074852  0.15118615 -0.36763847  0.09410439  0.02828674
  -0.09761526 -0.19148175 -0.08936858 -0.06899396  0.37643296 -0.28099838
  -0.02264663 -0.1597108  -0.18103924  0.14175169  0.11268579 -0.29266816
   0.01243196  0.00828274 -0.23055     0.02673402 -0.02587469  0.08534768
  -0.0561175  -0.21656604  0.08834951  0.10271733  0.02986447 -0.20377319
  -0.19586511 -0.03471686 -0.19465168 -0.11472059  0.03446077  0.28680608
  -0.00844208 -0.16234563  0.1778495  -0.429411    0.32706645 -0.01579501
   0.10330383 -0.03996562  0.2395629   0.08974864  0.07472736 -0.08321564
   0.00728825 -0.14286071 -0.06557172 -0.18970609 -0.23013942  0.18367453
   0.23298743  0.05290146 -0.01464603 -0.19841579  0.11357484 -0.07782242
  -0.02343861 -0.02729116 -0.02502235 

[[-2.56558925e-01  8.87655541e-02 -7.77638797e-03 -1.42157733e-01
  -2.33381435e-01 -3.04624856e-01 -1.32336959e-01 -7.05014989e-02
  -2.23391920e-01  1.01428866e-01 -5.47965840e-02  1.36711583e-01
  -1.02891862e-01  3.26009482e-01  6.64627030e-02  1.74821675e-01
   6.21688366e-03 -1.90523788e-01  9.03118774e-02  1.53130263e-01
   2.10326687e-01 -3.66621971e-01  1.51210308e-01 -9.65276733e-04
  -1.57433614e-01 -2.24194482e-01  1.14290044e-04 -4.51233797e-02
   2.70061493e-01 -2.62516379e-01 -6.04797974e-02 -2.21192554e-01
  -2.10236296e-01  2.47748811e-02 -6.03790209e-03 -3.12184900e-01
   1.03228062e-01  2.51616500e-02 -2.63117850e-01  5.51971346e-02
  -7.25644231e-02  7.26721138e-02 -2.13084053e-02 -1.20624654e-01
   9.37403813e-02  8.78987461e-02  8.07948560e-02 -6.93729073e-02
  -2.14779183e-01 -1.98646151e-02 -1.03923835e-01 -5.66224791e-02
   5.64039573e-02  2.91658610e-01  6.01032525e-02 -1.22641236e-01
   1.54379636e-01 -3.82542402e-01  3.82635623e-01  4.41094190e-02
   1.34925

  -0.32429415 -0.01713864 -0.17727058  0.23380606  0.04388976  0.12626918]]
Range: from 0 to 21 
[13]
Range: from 21 to 42 
[13, 36]
Range: from 42 to 63 
[13, 36, 58]
Range: from 63 to 84 
[13, 36, 58, 76]
Action  [0.3, 0.5, 0.6, 0.3]
[[-1.33045882e-01 -4.48364019e-03  1.33126616e-01 -8.52819681e-02
  -2.13613123e-01 -2.49368921e-01 -3.13289016e-01 -9.27541405e-02
  -2.06598878e-01  1.83186859e-01  2.41882354e-02  2.19656467e-01
   9.19666700e-03  3.46978962e-01  4.95253690e-02  1.25565439e-01
  -1.07268304e-01 -1.85304582e-01 -3.18821743e-02  2.19958812e-01
   1.99216291e-01 -3.06822956e-01  5.96733317e-02 -6.29384220e-02
  -7.60654919e-03 -2.09070221e-01 -6.72036335e-02 -7.69813880e-02
   4.91949260e-01 -1.99214458e-01 -6.66357875e-02 -2.00213403e-01
  -1.95519030e-01  1.89643130e-01  8.50479156e-02 -3.28462511e-01
   2.35919207e-02  5.94631657e-02 -1.96747094e-01 -1.98402740e-02
  -1.03361234e-01  4.77354601e-02 -7.11889863e-02 -3.53948355e-01
   1.24512114e-01  1.17720455e-01 -9.3